In [1]:
import pandas as pd

In [12]:
chiplist = ["CHIP1","CHIP2"]
poslist = ["POS1","POS2"]
offsetlist = ["Offset0","Offset1","Offset2","Offset3","Offset4","Offset5","Offset6","Offset7"]

mind = pd.MultiIndex.from_product([offsetlist, ['obnum','q1','u1','dolp1','phi_l1','q1prime','u1prime','errq1','erru1','q2','u2','dolp2','phi_l2','q2prime','u2prime','errq2','erru2','xceno','yceno','xcene','ycene']],
                           names=['offset', 'datapoint'])
mind_insideout = pd.MultiIndex.from_frame(pd.DataFrame.from_dict(mind.to_frame().reset_index(drop=True).T.reset_index(drop=True).T.to_dict()))
mind_insideout

MultiIndex([('Offset0',   'obnum'),
            ('Offset0',      'q1'),
            ('Offset0',      'u1'),
            ('Offset0',   'dolp1'),
            ('Offset0',  'phi_l1'),
            ('Offset0', 'q1prime'),
            ('Offset0', 'u1prime'),
            ('Offset0',   'errq1'),
            ('Offset0',   'erru1'),
            ('Offset0',      'q2'),
            ...
            ('Offset7',   'dolp2'),
            ('Offset7',  'phi_l2'),
            ('Offset7', 'q2prime'),
            ('Offset7', 'u2prime'),
            ('Offset7',   'errq2'),
            ('Offset7',   'erru2'),
            ('Offset7',   'xceno'),
            ('Offset7',   'yceno'),
            ('Offset7',   'xcene'),
            ('Offset7',   'ycene')],
           names=[0, 1], length=168)

In [4]:
[kw == 3 for kw in [1,2,3,4]]

[False, False, True, False]

In [5]:
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import dash_bootstrap_components as dbc
import dash
from dash import html, dcc, Input, Output, callback, State
from astropy.stats import sigma_clipped_stats as scs
from dash.exceptions import PreventUpdate
import warnings
import time
from plotly.subplots import make_subplots
warnings.filterwarnings("ignore", message = ".*FITS standard.*")
warnings.filterwarnings("ignore", message = ".*RADECSYS.*")
warnings.filterwarnings("ignore", message = ".*invalid value encountered in sqrt.*")
warnings.filterwarnings("ignore", message = ".*Mean of empty slice.*")
warnings.filterwarnings("ignore", message = ".*scalar divide.*")
warnings.filterwarnings("ignore", message = ".*PerformanceWarning.*")
warnings.filterwarnings("ignore", message = ".*DataFrame is highly fragmented.*")
warnings.filterwarnings("ignore", message = ".*Input data contains invalid values.*")

chiplist = ["CHIP1","CHIP2"]
poslist = ["POS1","POS2"]
offsetlist = ["Offset0","Offset1","Offset2","Offset3","Offset4","Offset5","Offset6","Offset7"]
tlist = ['2','2p25','2p5','2p75','3']
# with open("dict_dfpoldata_rot_2p25.pkl",'rb') as inp:
#     dict_dfpoldata_rot_2p25 = pickle.load(inp)
# with open('./df_poldata_2p25_exp_clean_forplotly.pkl','rb') as inp:
#     df_poldata_2p25_final = pickle.load(inp)
# with open('./binned_stats_forplot.pkl','rb') as inp:
#     df_binned_stats_forplot = pickle.load(inp)



######################## defs #############################################
####################### ROTATE ############################################            
def rotate(qutheta,theta_key = 'theta'):
    rotated_q,rotated_u = np.around(np.dot(rotmat(qutheta[('across_offsets',theta_key)]),\
                                           qutheta.loc[pd.IndexSlice[:,['q_scm','u_scm']]]),decimals = 10)
    return rotated_q,rotated_u
def rotmat(theta):
    return [[np.cos(2*theta),np.sin(2*theta)],[-np.sin(2*theta),np.cos(2*theta)]]
def rotate_exp(df_to_be_rotated,tk = 'theta'):
    rotation_mind = pd.MultiIndex.from_product([['across_offsets'],['rotated_q','rotated_u']])
    rotateddf = df_to_be_rotated.loc[:,pd.IndexSlice[:,['q_scm','u_scm',tk]]].apply(rotate,axis = 1,result_type = 'expand',**{'theta_key' : tk})
    rotateddf.columns = rotation_mind
    return rotateddf

####################### CLEAN & EXPAND DF #################################
def expand_df(combined_df,sig = 2):
    # print(sig)

    # # combine all positions
    # df_combined_frag = pd.concat([pd.concat([combined_df[chip][pos]], keys=[(chip,pos)], names=['CHIP','POS','row_index']) for chip in chiplist for pos in poslist], axis = 0)

    # calculate ellipticity,q/u_scm (sub inlau, inlaq) and avg phi_l
    combined_df.loc[:,('across_offsets','ellipticity')] = combined_df.loc[:,('across_offsets','b')]/combined_df.loc[:,('across_offsets','a')]

    qstats = scs(combined_df.loc[:,pd.IndexSlice[:,['q1','q2']]],axis = 1, sigma = sig)
    ustats = scs(combined_df.loc[:,pd.IndexSlice[:,['u1','u2']]],axis = 1, sigma = sig)
    combined_df.loc[:,('across_offsets','q_scm')] = qstats[0]
    combined_df.loc[:,('across_offsets','u_scm')] = ustats[0]

    combined_df.loc[:,('across_offsets','q_scerr')] = qstats[2]
    combined_df.loc[:,('across_offsets','u_scerr')] = ustats[2]

    combined_df.loc[:,('across_offsets','q_weights')] = 1/(qstats[2]**2)
    combined_df.loc[:,('across_offsets','u_weights')] = 1/(ustats[2]**2)

    combined_df.loc[:,('across_offsets','avg_phil')] = 0.5 * np.arctan2(combined_df.loc[:,('across_offsets','u_scm')],combined_df.loc[:,('across_offsets','q_scm')])

    return combined_df.copy()
def droprows_bin(df_to_be_cleaned,stars = True,ellipticity = True,npixthresh = 20,higherror = None):
    if stars:
        df_to_be_cleaned = df_to_be_cleaned.drop(df_to_be_cleaned[df_to_be_cleaned[('across_offsets','object_type')]=='star'].index,inplace = False)
    if higherror is not None:
        df_to_be_cleaned = df_to_be_cleaned.drop(df_to_be_cleaned[df_to_be_cleaned[('across_offsets','q_scerr')]>higherror].index,inplace = False)
        df_to_be_cleaned = df_to_be_cleaned.drop(df_to_be_cleaned[df_to_be_cleaned[('across_offsets','u_scerr')]>higherror].index,inplace = False)
    if ellipticity:
        df_to_be_cleaned = df_to_be_cleaned.drop(df_to_be_cleaned[df_to_be_cleaned[('across_offsets','ellipticity')].isnull()].index,inplace = False)
    if npixthresh is not None:
        df_to_be_cleaned = df_to_be_cleaned.drop(df_to_be_cleaned[df_to_be_cleaned[('across_offsets','npix')]<npixthresh].index,inplace = False)
    return df_to_be_cleaned.copy()
def add_bins(df_to_be_binned,binsize = 100):
    df_to_be_binned = df_to_be_binned.sort_values(('across_offsets','ellipticity'),ascending = False)
    df_to_be_binned.loc[:,('across_offsets','ell_bins')] = np.floor(np.arange(df_to_be_binned.shape[0])/binsize).astype(int).astype(str)
    return df_to_be_binned.copy()

####################### PLOT SUPPORT ######################################
def get_rls(cleaned_df, column_name = 'q_scerr'):
    # logseries = np.log(cleaned_df[('across_offsets',column_name)]/np.nanmax(cleaned_df[('across_offsets',column_name)]))
    logseries = np.log(cleaned_df[column_name]/np.nanmax(cleaned_df[column_name]))
    return (logseries/np.nanmin(logseries))

####################### WEIGHTED CALCS ####################################
def expand_averages(rotated_df_for_exp):
    # print(rotated_df_for_exp['across_offsets'])
    rotated_df_for_exp.loc[:,('across_offsets','q_x_weights')] = rotated_df_for_exp.loc[:,('across_offsets','rotated_q')]*rotated_df_for_exp.loc[:,('across_offsets','q_weights')]
    rotated_df_for_exp.loc[:,('across_offsets','u_x_weights')] = rotated_df_for_exp.loc[:,('across_offsets','rotated_u')]*rotated_df_for_exp.loc[:,('across_offsets','u_weights')]
    binned_stats = rotated_df_for_exp['across_offsets'].groupby('ell_bins')[['q_x_weights','q_weights','u_x_weights','u_weights']].sum()
    # binned_stats.loc[:,'q_wa'] =binned_stats['q_x_weights']/binned_stats['q_weights']
    # binned_stats.loc[:,'u_wa'] =binned_stats['u_x_weights']/binned_stats['u_weights']
    binned_stats['q_wa'] =binned_stats['q_x_weights']/binned_stats['q_weights']
    binned_stats['u_wa'] =binned_stats['u_x_weights']/binned_stats['u_weights']
    rotated_df_for_exp.loc[:,('across_offsets','binavg_q')] = binned_stats['q_wa'].loc[rotated_df_for_exp.loc[:,('across_offsets','ell_bins')].to_numpy()].to_numpy()
    rotated_df_for_exp.loc[:,('across_offsets','binavg_u')] = binned_stats['q_wa'].loc[rotated_df_for_exp.loc[:,('across_offsets','ell_bins')].to_numpy()].to_numpy()
    rotated_df_for_exp.loc[:,('across_offsets','weighted_q_diff_sq')] = ((rotated_df_for_exp.loc[:,('across_offsets','rotated_q')]-rotated_df_for_exp.loc[:,('across_offsets','binavg_q')])**2)*(rotated_df_for_exp.loc[:,('across_offsets','q_weights')])
    rotated_df_for_exp.loc[:,('across_offsets','weighted_u_diff_sq')] = ((rotated_df_for_exp.loc[:,('across_offsets','rotated_u')]-rotated_df_for_exp.loc[:,('across_offsets','binavg_u')])**2)*(rotated_df_for_exp.loc[:,('across_offsets','u_weights')])
    binned_stats_err = rotated_df_for_exp['across_offsets'].groupby('ell_bins')[['weighted_q_diff_sq','q_weights','weighted_u_diff_sq','u_weights']].sum()
    binned_stats_err['weighted_err_q'] =np.sqrt(binned_stats_err['weighted_q_diff_sq']/binned_stats_err['q_weights'])
    binned_stats_err['weighted_err_u'] =np.sqrt(binned_stats_err['weighted_u_diff_sq']/binned_stats_err['u_weights'])
    binned_stats['x_center'] = rotated_df_for_exp['across_offsets'].groupby('ell_bins')['ellipticity'].median()
    
    binned_stats = pd.concat([binned_stats,binned_stats_err[['weighted_err_q','weighted_err_u']]],axis = 1)
    
    return binned_stats

###########################################################################

####################### setup init ########################################
init_t,init_npix,init_errorthresh,init_bins,init_sigexp,init_tkinput = '2p25',None,None,100,2,'theta'
init_df_statedict = {
    't':'2p25',
    'npix':None,
    'errorthresh':None,
    'bins':100,
    'sigexp':2,
    'tk':'theta',
}

####################### get data ##########################################
def calc_scstats_and_avgphil(t,sig_exp):
    # get_df
    basedf_to_expand = dict_basedf[t]
    # create scm columns
    exp = expand_df(basedf_to_expand,sig_exp)
    return exp
def rotate_df(df_exp,tk_input = 'theta'):
    # rotate
    rotated_qu = rotate_exp(df_exp, tk = tk_input)
    df_exp[('across_offsets','rotated_q')],df_exp[('across_offsets','rotated_u')] = \
        rotated_qu[('across_offsets','rotated_q')],rotated_qu[('across_offsets','rotated_u')]
    return df_exp
def clean_and_bin(exp_df,npix,errorthresh,bins):
    # clean and bin
    exp_clean = droprows_bin(exp_df, npixthresh=npix, higherror = errorthresh)
    # calculate binned stats
    binned_exp_clean = add_bins(exp_clean,bins)
    binned_stats = expand_averages(binned_exp_clean)
    return binned_exp_clean, binned_stats
def read_from_store_wrapper(dicts,rows = None,cols = None,levels = 2):
    # if dict of dicts, then iterate (in the case of qu dicts)
    if levels == 2:
        return {k:pd.DataFrame.from_dict(dict_df) for k,dict_df in dicts.items()}
    # if multiindex cols and rows, add them to the data
    elif levels == 1:
        df = pd.DataFrame.from_dict(dicts)
        if cols is not None:
            df.columns = cols
        else:
            print("columns not added")
        if rows is not None:
            df.index = rows
        else:
            print("rows not added")
        return df
def jsonify(df):
    df_reset = df.reset_index(drop = True).T.reset_index(drop=True).T
    return df_reset.to_dict()
def get_qudf(c,p,ri,df_reset):
    qs = df_reset.loc[(c,p,ri),pd.IndexSlice[:,['q1','q2','errq1','errq2']]].unstack(level = 1)
    us = df_reset.loc[(c,p,ri),pd.IndexSlice[:,['u1','u2','erru1','erru2']]].unstack(level = 1)
    return {'q':qs, 'u':us}

def get_basedf(t_pull):
    with open("../dict_dfpoldata_rot_"+t_pull+".pkl",'rb') as inp:
        basedf = pickle.load(inp)
    # combine all positions
    df_combined_frag = pd.concat([pd.concat([basedf[chip][pos]], keys=[(chip,pos)], names=['CHIP','POS','row_index'])\
                                  for chip in chiplist for pos in poslist], axis = 0)
    return df_combined_frag

dict_basedf = {t:get_basedf(t_pull = t) for t in tlist}

###################################### Figure Functions ######################################################
def make_stats_scatter(c,p,ri,df_reset):
    figspread = px.scatter(
        df_reset.reset_index(),
        x='q_scm',
        y ='u_scm',
        color='ell_bins',
        custom_data=['CHIP','POS','row_index'],
        # opacity = get_rls(df_poldata_2p25_exp_clean_forplotly),
        height=620,
        # width = 620,
        )
    # figspread.update_layout(xaxis=dict(scaleanchor='y', scaleratio=1))
    figspread.layout.xaxis.range = [-0.3,0.3]
    figspread.layout.yaxis.range = [-0.3,0.3]
    figspread.add_hline(
            # x0=0,
            y=df_reset.loc[(c,p,ri),'u_scm'],#.iloc[0],
            # x1=7,
            # y1 = stats[0],
            # line=dict(color='Black',dash = 'dash',line_width = 1),
            # color='Black',
            line_dash = 'solid',
            line_width = 0.5,
            # xref='x',
            # yref='y',
        )
    figspread.add_vline(
            # x0=0,
            x=df_reset.loc[(c,p,ri),'q_scm'],#.iloc[0],
            # x1=7,
            # y1 = stats[0],
            # line=dict(color='Black',dash = 'dash',line_width = 1),
            # color='Black',
            line_dash = 'solid',
            line_width = 0.5,
            name = 'vline',
            # xref='x',
            # yref='y',
        )

    figspread.update_layout(paper_bgcolor='rgba(0,0,0,0)')#,plot_bgcolor='rgba(0,0,0,0)',
    figspread.update_layout(legend=dict(
        orientation='h',
        yanchor="bottom",
        y=-0.20,
        xanchor="left",
        x=-0.05
    ))
    figspread.update_layout(margin=dict(l=20, r=20, t=20, b=20),)
    figspread.update_xaxes(showline=True, linewidth=4, linecolor='purple')
    figspread.update_yaxes(showline=True, linewidth=4, linecolor='red')
    return figspread
def make_individ_stats_fig(qu,c,p,ri,sig,qudf_dict,df_reset):
    qudf = qudf_dict[qu]
    # print(qudf['Offset0':'Offset7'])
    scm = df_reset.loc[(c,p,ri),qu+'_scm']#.iloc[0]
    scerr = df_reset.loc[(c,p,ri),qu+'_scerr']#.iloc[0]
    # print(scm,scerr)
    fig = px.scatter(qudf,
        x=offsetlist,
        y = qu + '1',
        error_y = 'err' + qu + '1',
        height = 310,
        # labels=dict(y="<"+str(qu)+">", color="Ellipticity bins"),
    )
    # fig1.show()
    fig2 = px.scatter(qudf,
        x=offsetlist,
        y = qu+'2',
        error_y = 'err' + qu + '2',
        color_discrete_sequence=['red'],
        height = 310,
        # custom_data=[qu,c,p,ri],
        # labels=dict(x="Offsets", y="<"+str(qu)+">", color="Ellipticity bins"),
    )
    if qu == 'u':
        fig.update_layout(xaxis_title="Offsets", yaxis_title = "<"+str(qu)+">")
    else:
        fig.update_layout(xaxis_title=None, yaxis_title = "<"+str(qu)+">")
    # fig.update()

    fig.add_trace(fig2['data'][0])
    fig.add_hline(
            # x0=0,
            y=0,
            # x1=7,
            # y1 = stats[0],
            # line=dict(color='Black',dash = 'dash',line_width = 1),
            # color='Black',
            line_dash = 'solid',
            line_width = 1,
            # xref='x',
            # yref='y',
        )
    fig.add_hline(
            # x0=0,
            y=scm,
            # x1=7,
            # y1 = stats[0],
            line=dict(color='darkgreen',dash = 'solid',),
            # color='Black',
            # line_dash = 'dashed',
            line_width = 0.5,
            # xref='x',
            # yref='y',
        )
    fig.add_annotation(
        x=6.5,
        y=scm,
        text = qu+"_scm = {}".format(np.around(scm,4)),
        # textposition = 'top center',
        showarrow=True,
        bgcolor = 'green',
        font = {'color':'white'},
        opacity = 0.5,
        xanchor="right",
        )
    fig.add_hrect(
            # x0=0,
            y0 = scm-(sig*scerr),
            # x1=7,
            y1 = scm+(sig*scerr),
            # line=dict(color=None,dash = 'dash'),
            fillcolor="turquoise",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
    fig.update_yaxes(range = [scm-(5*scerr),scm+(5*scerr)])
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)') #,plot_bgcolor='rgba(0,0,0,0)',
    fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)
    return fig
def make_trendplot(df_reset,dfstats):
    fig = make_subplots(rows=1, cols=2, shared_yaxes='rows',subplot_titles=("rotated_q vs. ellipticity","rotated_u vs. ellipticity"), horizontal_spacing=0.07)
    figq = px.scatter(
        df_reset,
        x='ellipticity',
        y ='rotated_q',
        color='ell_bins',
        # facet_row='ell_bins',
        # opacity = get_rls(df_reset),
        # trendline = 'lowess',
        # trendline_options = dict(frac=1),
        # height=1500,
    )
    # fig1.show()

    figu = px.scatter(
        df_reset,
        x='ellipticity',
        y ='rotated_u',
        color='ell_bins',
        # title=""
        # opacity = get_rls(df_reset),
        # trendline = 'lowess',
        # trendline_options = dict(frac=1),
        # height=50,
        # showlegend = False
    )

    fig_qyerr = px.scatter(dfstats, x="x_center", y="q_wa",error_y="weighted_err_q",color_discrete_sequence = ['black'])
    fig_uyerr = px.scatter(dfstats, x="x_center", y="u_wa",error_y="weighted_err_u",color_discrete_sequence = ['black'])

    for trace in range(len(figq["data"])):
        fig.add_trace(figq["data"][trace], col = 1, row = 1,)
    fig.add_trace(fig_qyerr["data"][0], col = 1, row = 1,)

    for trace in range(len(figu["data"])):
        figu["data"][trace]['showlegend'] = False
        fig.add_trace(figu["data"][trace], col = 2, row = 1,)
    fig.add_trace(fig_uyerr["data"][0], col = 2, row = 1,)

    # for trace in range(len(figspread["data"])):
    #     # figspread["data"][trace]['showlegend'] = False
    #     fig.add_trace(figspread["data"][trace], col = 1, row = 2,)

    fig.update_yaxes(range=[-0.3, 0.3], dtick=0.05,col = 1,row = 1)
    fig.update_yaxes(range=[-0.3, 0.3], dtick=0.05,col = 2,row = 1)
    fig.update_xaxes(range=[1,0.3], dtick=0.1,col = 1,row = 1)
    fig.update_xaxes(range=[1,0.3], dtick=0.1,col = 2,row = 1)
    # fig.update_layout(template = "plotly_dark")
    fig.add_hline(y = 0,col = 1, row = 1)
    fig.add_hline(y = 0,col = 2, row = 1)
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)') #,plot_bgcolor='rgba(0,0,0,0)',
    fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
    )
    # fig.update_layout(
    #     # font_family="Courier New",
    #     font_color="white",
    #     # title_font_family="Times New Roman",
    #     title_font_color="white",
    #     legend_title_font_color="white"
    # )
    # fig.update_layout(title_text="polarimetry in the plane of the object") #height=600, width=900, 
    return fig

###################################### get init datasets #####################################################
# tm2 = time.time()
init_df_stats = calc_scstats_and_avgphil(init_t,init_sigexp)
# tm1 = time.time()
# print("stats took {} seconds".format(tm1-tm2))

# t0 = time.time()
init_df_rotated = rotate_df(init_df_stats,init_tkinput)
# t1 = time.time()
init_df_filtered, init_df_binned_stats = clean_and_bin(init_df_rotated,init_npix,init_errorthresh,init_bins)
# t2 = time.time()
# print("rotation took {} seconds".format(t1-t0))
# print("cleaning and binning took {} seconds".format(t2-t1))

init_chip,init_pos,init_rowindex = init_df_filtered.iloc[0].name
init_object_statedict = {
    'chip':init_chip,
    'pos':init_pos,
    'row_index':init_rowindex,
}
# init_df_statedict = {
#     't':init_t,
#     'sigexp':init_sigexp,
#     'npix':init_npix,
#     'errorthresh':init_errorthresh,
#     'bins':init_bins,
#     'tk':init_tkinput,
# }

init_rotated_col_names = init_df_rotated.columns
init_rotated_row_names = init_df_rotated.index
init_filtered_col_names = init_df_filtered.columns
init_filtered_row_names = init_df_filtered.index
# t5 = time.time()
init_qudict = get_qudf(init_chip, init_pos, init_rowindex, init_df_filtered)
# t6 = time.time()

# print("qudicts took {} seconds".format(t6-t5))

# # make init plots
init_fig_spread = make_stats_scatter(init_chip, init_pos, init_rowindex, init_df_filtered['across_offsets'])
init_fig_rotated = make_trendplot(init_df_filtered['across_offsets'],init_df_binned_stats)
init_fig_q = make_individ_stats_fig('q', init_chip, init_pos, init_rowindex, init_sigexp, init_qudict, init_df_filtered['across_offsets'])
init_fig_u = make_individ_stats_fig('u', init_chip, init_pos, init_rowindex, init_sigexp, init_qudict, init_df_filtered['across_offsets'])
    
#################################################
###################################################
# dash.register_page(__name__, path='/')

def serve_layout():
    return dbc.Container([
        # local stores
        # dcc.Store(id = 'global_df_rotated', data = init_df_rotated),
        # dcc.Store(id = 'global_rotated_rows', data = init_rotated_row_names),
        # dcc.Store(id = 'global_rotated_cols', data = init_rotated_col_names),
        # dcc.Store(id = 'global_df_filtered', data = init_df_filtered),
        # dcc.Store(id = 'global_filtered_rows', data = init_filtered_row_names),
        # dcc.Store(id = 'global_filtered_cols', data = init_filtered_col_names),
        # dcc.Store(id = 'global_qudict', data = init_qudict),
        # dcc.Store(id = 'global_df_binned', data = init_df_binned_stats),
        # dcc.Store(id = 'global_df_statedict', data = init_df_statedict),
        # dcc.Store(id = 'global_object_statedict', data = init_object_statedict),

        # dcc.Store(id = 'global_df_rotated',),
        # dcc.Store(id = 'global_rotated_rows',),
        # dcc.Store(id = 'global_rotated_cols',),
        # dcc.Store(id = 'global_df_filtered',),
        # dcc.Store(id = 'global_filtered_rows',),
        # dcc.Store(id = 'global_filtered_cols',),
        # dcc.Store(id = 'global_qudict',),
        # dcc.Store(id = 'global_df_binned',),
        # dcc.Store(id = 'global_df_statedict',),
        # dcc.Store(id = 'global_object_statedict',),

        # title
        dbc.Row([
            dbc.Col([
                html.H1("Object Statistics",
                    style = {'textAlign': 'center'}),
            ], width = 12, class_name = "bg-light border p-2 border-2"),
            
        ],class_name= "m-2 border-0"),
        
        # df buttons
        dbc.Row([
            dbc.Col(['apperture scale: ',
                dcc.Dropdown(id='t_input', options = tlist,placeholder = 'scale value for data retireval', value = init_t),
            ], width = 2),
            dbc.Col(["sigma: ",
                dcc.Dropdown(id='sig_input', options = [1,2,3],placeholder = 'sigma value for sigma clipping', value = init_sigexp),
            ], width = 2),
            dbc.Col(["Min pix for object: ",
                dcc.Input(id='npix_input', type = 'number', placeholder = 'npix : number or None', debounce = True, required = True, value = init_npix),
            ], width = 2),
            dbc.Col(["Maximum error for object: ",
                dcc.Input(id='errorthresh_input', type = 'number', placeholder = 'high error threshhold', debounce = True, required = True, value = init_errorthresh),
            ], width = 2),
            dbc.Col(["Bin size: ",
                dcc.Input(id='bins_input', type = 'number',placeholder = 'Size of each bin', debounce = True, required = True, value = init_bins),
            ], width = 2),
            dbc.Col(['rotation angle: ',
                dcc.Dropdown(id='tk_input', options = ['theta','avg_phil'], placeholder = 'angle used for rotating', value = init_tkinput),
            ], width = 1),
            dbc.Col([
                html.Button('Refresh', id='refresh_button',n_clicks=0),
            ], width = 1),
        ], style = {'textAlign': 'center'}, class_name= "bg-light m-2 border-0"),
        
        # object plots
        dbc.Row([
            dbc.Col([
                dcc.Graph(figure = init_fig_spread, id='spread',)
            ],width = 6, style={'margin':'auto','textAlign':'left',}, className = "bg-light p-1 m-2 justify-content-end"), #style = {'display': 'inline-block'} ,"background-color": "cyan"  'height':'50vh'
            dbc.Col([
                dbc.Row([
                    dcc.Graph(figure = init_fig_q, id='polarimetry_q',)
                ],className = "bg-light border-start border-5 border-info p-1 m-1"), #style = {'display': 'inline-block'} 'height': '25vh'
                dbc.Row([
                    dcc.Graph(figure = init_fig_u, id='polarimetry_u',)
                ],className = "bg-light border-start border-5 border-danger p-1 m-1"), #style = {'display': 'inline-block'} 'height': '25vh'
            ],width = 5,class_name = "gy-0 p-1 m-1 d-inline justify-content-start"),
        ], class_name = "p-1 m-2 justify-content-center"),

        # object buttons
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(id='chip_input', options = chiplist, placeholder = "enter object chip"),
            ], width = 3),
            dbc.Col([
                dcc.Dropdown(id='pos_input', options = poslist, placeholder = "enter object pos"),
            ], width = 3),
            dbc.Col([
                dcc.Input(id='row_index_input', type = 'number', placeholder = "enter object row index", debounce = True, required = True),
            ], width = 3),
            dbc.Col([
                html.Button('Submit', id='submit_button', n_clicks=0),
            ], width = 3),
        ],style = {'textAlign': 'center'}),
        
        # rotated plots
        dbc.Row([html.H3(["POLARIMETRY IN THE PLANE OF THE OBJECT"], className = "p-3 border border-4"),
            dcc.Graph(figure = init_fig_rotated, id='rotated_quvsell',style = {'height': '45vh', 'textAlign':'center'})
        ],style = {'textAlign':'center'}, class_name = "bg-light p-0 m-0 gx-2"), #style = {'display': 'inline-block'}

        # test row
        dbc.Row(html.Div(
            children = "testing output from clickdata",
            id = 'test')),
    ]) #, class_name = "bg-secondary bg-opacity-25 gx-2"


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = serve_layout

# layout = serve_layout

@callback(
    # # update figures
    Output('polarimetry_q','figure'),
    Output('polarimetry_u','figure'),
    Output('spread','figure'),
    Output('rotated_quvsell','figure'),

    # # update stores
    Output('global_df_rotated','data'),
    Output('global_rotated_rows','data'),
    Output('global_rotated_cols','data'),
    Output('global_df_filtered','data'),
    Output('global_filtered_rows','data'),
    Output('global_filtered_cols','data'),
    Output('global_qudict','data'),
    Output('global_df_binned','data'),
    Output('global_df_statedict','data'),
    Output('global_object_statedict','data'),

    # update dropdowns/inputs
    Output('chip_input','value'),
    Output('pos_input','value'),
    Output('row_index_input','value'),
    Output('spread','clickData'),

    #######################################
    
    # get click data
    Input('spread','clickData'),
    Input('refresh_button', 'n_clicks'),
    Input('submit_button', 'n_clicks'),

    #######################################

    # read df states
    State('t_input','value'),
    State('sig_input','value'),
    State('npix_input','value'),
    State('errorthresh_input','value'),
    State('bins_input','value'),
    State('tk_input','value'),

    # read object states
    State('chip_input','value'),
    State('pos_input','value'),
    State('row_index_input','value'),

    # read stores
    State('global_df_rotated','data'),
    State('global_rotated_rows','data'),
    State('global_rotated_cols','data'),
    State('global_df_filtered','data'),
    State('global_filtered_rows','data'),
    State('global_filtered_cols','data'),
    State('global_qudict','data'),
    State('global_df_binned','data'),
    State('global_df_statedict','data'),
    State('global_object_statedict','data'),

    # callback setup
    # prevent_initial_call = 'True',
    # suppress_callback_exceptions=True,#data_clickcount, 
)   
def update_figs(spread_clickdata,refresh_clicks,submit_clicks,\
                t_input, sig_input, npix_input, errorthresh_input, bins_input, tk_input,\
                chip_input, pos_input, row_index_input,\
                global_df_rotated, global_rotated_rows, global_rotated_cols, global_df_filtered, global_filtered_rows, global_filtered_cols, global_qudict, global_df_binned, global_df_statedict, global_object_statedict):

    # conditionals to get all data needed for plots
    print("triggered by element with id : {}".format(dash.callback_context.triggered_id))
    print("triggered: {}".format(format(dash.callback_context.triggered)))

    # firstrun: get df, expand, and plot
    if dash.callback_context.triggered_id is None:
        print("all initial setup is done in the main file")
        raise PreventUpdate
        chip_plot,pos_plot,rowindex_plot = chip_input, pos_input, row_index_input

    #     # if new filtered df required: get basedf and apply filters
    elif (dash.callback_context.triggered_id == 'refresh_button') and (refresh_clicks > 0):
        print('refresh clicked',refresh_clicks)#data_clickcount,

        # run calcs
        df_stats = calc_scstats_and_avgphil(t_input,sig_input)
        df_rotated = rotate_df(df_stats,tk_input)
        df_filtered, df_binned_stats = clean_and_bin(df_rotated,npix_input,errorthresh_input,bins_input)
        chip_plot,pos_plot,rowindex_plot = df_filtered.iloc[0].name
        # spread_clickdata_plot = spread_clickdata
        qudict = get_qudf(chip_plot, pos_plot, rowindex_plot, df_filtered)

        # prep data for store
        rotated_col_names = df_rotated.columns
        rotated_row_names = df_rotated.index
        filtered_col_names = df_filtered.columns
        filtered_row_names = df_filtered.index
        object_statedict = {
            'chip':chip_plot,
            'pos':pos_plot,
            'row_index':rowindex_plot,
        }
        df_statedict = {
            't':t_input,
            'sigexp':sig_input,
            'npix':npix_input,
            'errorthresh':errorthresh_input,
            'bins':bins_input,
            'tk':tk_input,
        }
        dict_df_rotated_reset = jsonify(df_rotated)
        dict_df_filtered_reset = jsonify(df_filtered)
        dict_qudict = {qu:df.to_dict() for qu,df in qudict.items()}
        dict_dfbinned = df_binned_stats.to_dict()

        ## make plots
        # fig_spread = make_stats_scatter(chip_plot, pos_plot, rowindex_plot, df_filtered['across_offsets'])
        # fig_rotated = make_trendplot(df_filtered['across_offsets'],df_binned_stats)
        # fig_q = make_individ_stats_fig('q', chip_plot, pos_plot, rowindex_plot, sig_input, qudict, df_filtered['across_offsets'])
        # fig_u = make_individ_stats_fig('u', chip_plot, pos_plot, rowindex_plot, sig_input, qudict, df_filtered['across_offsets'])

        # object changed via submit
    elif (dash.callback_context.triggered_id == 'submit_button') and (submit_clicks > 0):
        print('submit clicked',submit_clicks)
        # df_poldata_exp_forplot, df_binned_stats_forplot = filter_data(t_input,npix_input,errorthresh_input,bins_input,sig_input,tk_input)
        # read filtered data from store
        # cols = store_cols
        # reset_df_for_store, df_binned_stats_forplot, qudict_for_store = pd.DataFrame.from_dict(store_reset_df),pd.DataFrame.from_dict(store_filtered_data_binned), store_qudict
        # reset_df_for_store.columns = store_cols
        # input_c, input_p, input_ri = input_chip,input_pos,input_row_index
        # qudict_for_store = get_qudf(input_c, input_p, input_ri, reset_df_for_store)
        # reflect the submit changes in the clickdata
        # spread_clickdata['points'][0]['customdata'] = [input_c, input_p, input_ri]
    elif (dash.callback_context.triggered_id == 'spread') and (spread_clickdata is not None):
        print('clickdata triggered', spread_clickdata)
        # # read filtered data from store
        # # cols = store_cols
        # # df_poldata_exp_forplot, df_binned_stats_forplot = filter_data(t_input,npix_input,errorthresh_input,bins_input,sig_input,tk_input)
        # reset_df_for_store, df_binned_stats_forplot, qudict_for_store = pd.DataFrame.from_dict(store_reset_df),pd.DataFrame.from_dict(store_filtered_data_binned), store_qudict
        # reset_df_for_store.columns = store_cols
        # input_c, input_p, input_ri = spread_clickdata['points'][0]['customdata']
        # qudict_for_store = get_qudf(input_c, input_p, input_ri, reset_df_for_store)
        # # print(input_c, input_p, input_ri)
    else:
        print("unknown combination of events")

    # chip_output, pos_output, row_index_output, spread_clickdata_output = chip_plot, pos_plot, rowindex_plot, spread_clickdata_plot

    # # make plots
    fig_spread = make_stats_scatter(chip_plot, pos_plot, rowindex_plot, df_filtered['across_offsets'])
    fig_rotated = make_trendplot(df_filtered['across_offsets'],df_binned_stats)
    fig_q = make_individ_stats_fig('q', chip_plot, pos_plot, rowindex_plot, sig_input, qudict, df_filtered['across_offsets'])
    fig_u = make_individ_stats_fig('u', chip_plot, pos_plot, rowindex_plot, sig_input, qudict, df_filtered['across_offsets'])

    return (fig_q, fig_u, fig_spread, fig_rotated,\
        dict_df_rotated_reset, rotated_col_names, rotated_row_names, dict_df_filtered_reset, filtered_col_names, filtered_row_names, dict_qudict,dict_dfbinned, df_statedict, object_statedict,\
        chip_plot, pos_plot, rowindex_plot, spread_clickdata)


# if __name__ == "__main__":
#     app.run(debug=True, port = 8005)

In [69]:
init_fig_u

In [53]:
init_qudict_for_store['q']

,q1,q2,errq1,errq2
Offset0,-4.157504,-0.400206,0.020194,0.017164
Offset1,-0.480500,0.954961,0.017572,0.018160
Offset2,1.192368,0.023594,0.016888,0.016820
Offset3,0.582002,-0.048538,0.017294,0.017640
Offset4,-1.122746,-0.363033,0.017282,0.017589
Offset5,0.499179,0.545057,0.021572,0.020046
Offset6,31.317196,-34.665017,0.026761,0.048636
Offset7,-0.399385,0.109637,0.021494,0.020380


In [11]:
init_rotated_df

Offset0                                           \
                       obnum        q1        u1     dolp1     phi_l1   
CHIP  POS  row_index                                                    
CHIP1 POS1 0            45.0 -2.794319  0.157248  2.798740  88.389566   
           1            47.0  0.088316  0.375817  0.386055  38.387819   
           2            48.0 -0.104939  0.140672  0.175502  63.361141   
           3            50.0  0.044482  0.001429  0.044505   0.920150   
           4            51.0  0.066725  0.034592  0.075159  13.701726   
...                      ...       ...       ...       ...        ...   
CHIP2 POS2 186           NaN -0.121888 -0.008226  0.122166 -88.069441   
           187           NaN -0.358755 -0.789499  0.867187 -57.218725   
           188           NaN -0.191025 -0.017412  0.191817 -87.395924   
           189           NaN  2.391850 -0.111639  2.394454  -1.336169   
           190           NaN -3.185229  0.619949  3.244999  84.493037   

                                                                        ...  \
                       q1prime   u1prime     errq1     erru1        q2  ...   
CHIP  POS  row_index                                                    ...   
CHIP1 POS1 0         -2.797856  0.070324  0.005060  0.005036  0.020054  ...   
           1          0.081041  0.377453  0.015723  0.017288 -0.340243  ...   
           2         -0.109284  0.137324  0.008914  0.008870  0.338893  ...   
           3          0.044496  0.000899  0.004781  0.004779  0.000070  ...   
           4          0.066796  0.034456  0.005185  0.005267  0.013653  ...   
...                        ...       ...       ...       ...       ...  ...   
CHIP2 POS2 186       -0.122050 -0.005322  0.009285  0.009199 -0.111802  ...   
           187       -0.358357 -0.789679  0.019381  0.018781  0.394078  ...   
           188       -0.189975 -0.026517       NaN       NaN -2.236779  ...   
           189        2.394435  0.009520  0.037739       NaN -1.315168  ...   
           190       -3.189308  0.598614       NaN       NaN -0.541802  ...   

                     across_offsets                                          \
                        ellipticity     q_scm     u_scm   q_scerr   u_scerr   
CHIP  POS  row_index                                                          
CHIP1 POS1 0               0.865471 -0.000775  0.012822  0.097486  0.083475   
           1               0.734559  0.058918  0.180512  0.225649  0.237614   
           2               0.780022 -0.000982  0.054737  0.097206  0.065072   
           3               0.840948 -0.004490 -0.008444  0.013100  0.029048   
           4               0.959569 -0.001197  0.017574  0.043371  0.039512   
...                             ...       ...       ...       ...       ...   
CHIP2 POS2 186             0.818433  0.009377  0.013438  0.195680  0.336756   
           187             0.823982 -0.111301 -0.067587  0.371080  0.488143   
           188             0.651548 -0.226358  0.104653  1.130947  1.120638   
           189             0.628231 -0.023254  0.353621  1.729723  2.299260   
           190             0.612616 -0.380735 -0.310771  3.984233  2.009193   

                                                                              
                        q_weights    u_weights  avg_phil rotated_q rotated_u  
CHIP  POS  row_index                                                          
CHIP1 POS1 0           105.224646   143.511918  0.815601 -0.012378 -0.003435  
           1            19.639608    17.711497  0.627651 -0.135039  0.133494  
           2           105.832020   236.166364  0.794363 -0.053203 -0.012905  
           3          5827.240567  1185.104973 -1.029775 -0.008812 -0.003715  
           4           531.611692   640.547719  0.819403  0.000860  0.017594  
...                           ...          ...       ...       ...       ...  
CHIP2 POS2 186          26.115925     8.818004  0.480779  0.015075 -0.006

In [10]:
init_df_filtered.index.get_loc(('CHIP2','POS2',int('133')))

549

In [15]:
df_binned_stats_forplot

,q_x_weights,q_weights,u_x_weights,u_weights,q_wa,u_wa,x_center,weighted_err_q,weighted_err_u
ell_bins,,,,,,,,,
0,-290.007547,577522.497605,-4367.942703,1.979142e+06,-0.000502,-0.002207,0.917713,0.007322,0.004845
1,28.497191,74865.096951,275.421091,1.279673e+05,0.000381,0.002152,0.874624,0.020537,0.018594
2,-35.869245,27145.917551,-171.155186,3.288262e+04,-0.001321,-0.005205,0.842907,0.032378,0.033946
3,-191.939501,53674.086063,-64.484762,5.109061e+04,-0.003576,-0.001262,0.811870,0.022802,0.028495
4,-644.473750,194137.553134,-338.814622,4.209649e+05,-0.003320,-0.000805,0.776113,0.015325,0.007443
5,298.618506,97815.525857,-44.983509,1.369982e+05,0.003053,-0.000328,0.739525,0.018626,0.020149
6,249.172719,73318.952998,-180.791497,5.046165e+04,0.003398,-0.003583,0.696169,0.015496,0.026145
7,-323.959167,181422.811628,-409.635946,1.421517e+05,-0.001786,-0.002882,0.634403,0.016161,0.014141
8,375.556142,44749.078986,-121.493930,6.689954e+04,0.008392,-0.001816,0.487451,0.024049,0.022776


In [39]:
def read_from_store_wrapper(dicts,rows = None,cols = None,levels = 2):
    # if dict of dicts, then iterate (in the case of qu dicts)
    if levels == 2:
        return {k:pd.DataFrame.from_dict(dict_df) for k,dict_df in dicts.items()}
    # if multiindex cols and rows, add them to the data
    elif levels == 1:
        df = pd.DataFrame.from_dict(dicts)
        if cols is not None:
            df.columns = cols
        else:
            print("columns not added")
        if rows is not None:
            df.index = rows
        else:
            print("rows not added")
        return df

In [46]:
read_from_store_wrapper(init_df_poldata_filtered.T.reset_index(drop=True).T.to_dict(),rows = init_row_names,cols = init_col_names,levels = 1)

Offset0                                               \
                       obnum          q1        u1       dolp1     phi_l1   
CHIP  POS  row_index                                                        
CHIP1 POS2 197           NaN   -4.157504  0.118880    4.159203  89.181065   
      POS1 164         364.0   -0.003346 -0.008322    0.008970 -55.952098   
      POS2 96          233.0    0.003616  0.003155    0.004798  20.551589   
           93          230.0    0.008567  0.007817    0.011598  21.189749   
      POS1 192         426.0   -0.006650  0.003108    0.007341  77.473077   
...                      ...         ...       ...         ...        ...   
      POS2 208           NaN -120.605554 -1.488820  120.614743 -89.646373   
CHIP2 POS1 79          153.0   -0.200699 -0.369060    0.420101 -59.268941   
      POS2 159           NaN    0.117994 -0.147961    0.189248 -25.714335   
CHIP1 POS2 99          241.0  -94.222444  0.217858   94.222696  89.933762   
CHIP2 POS2 155           NaN   -6.420582  2.074989    6.747551  81.045160   

                                                                           \
                         q1prime   u1prime     errq1     erru1         q2   
CHIP  POS  row_index                                                        
CHIP1 POS2 197         -4.156860  0.139599  0.020194  0.016432  -0.400206   
      POS1 164         -0.003466 -0.008273  0.001773  0.001771  -0.002933   
      POS2 96           0.003618  0.003152  0.001466  0.001469   0.002537   
           93           0.008705  0.007664  0.000370  0.000363  -0.004920   
      POS1 192         -0.006629  0.003152  0.003236  0.003264   0.014927   
...                          ...       ...       ...       ...        ...   
      POS2 208       -120.610547 -1.006045       NaN  0.031260   3.876188   
CHIP2 POS1 79          -0.193903 -0.372675  0.005245  0.027293 -16.296551   
      POS2 159          0.120133 -0.146229  0.014212  0.015464  -0.824469   
CHIP1 POS2 99         -94.131353 -4.147865  0.041330  0.002238  -0.589207   
CHIP2 POS2 155         -6.496294  1.824173  0.014560  0.011982   0.198377   

                      ... across_offsets                               \
                      ...       avg_phil rotated_q rotated_u ell_bins   
CHIP  POS  row_index  ...                                               
CHIP1 POS2 197        ...       0.441620  0.109451  0.074506        0   
      POS1 164        ...       1.348854 -0.000534  0.001728        0   
      POS2 96         ...       0.931377 -0.000722  0.002811        0   
           93         ...      -0.072418  0.001245 -0.002900        0   
      POS1 192        ...       0.817159  0.000537  0.001656        0   
...                   ...            ...       ...       ...      ...   
      POS2 208        ...       0.647398  0.504375  0.911635        8   
CHIP2 POS1 79         ...       0.656325 -0.739345  0.730013        8   
      POS2 159        ...       1.407754 -0.815850 -0.453320        8   
CHIP1 POS2 99         ...       0.339164 -2.378193 -0.463047        8   
CHIP2 POS2 155        ...       1.481293  0.573842 -1.115775        8   

                                                                   \
                     q_x_weights  u_x_weights  binavg_q  binavg_u   
CHIP  POS  row_index                                                
CHIP1 POS2 197          0.279830     1.142298 -0.000502 -0.000502   
      POS1 164        -28.517755    38.548977 -0.000502 -0.000502   
      POS2 96         -48.149906   293.878244 -0.000502 -0.000502   
           93          95.536068 -3953.186827 -0.000502 -0.000502   
      POS1 192          1.690906     8.544007 -0.000502 -0.000502   
...                          ...          ...       ...       ...   
      POS2 208          0.066144     0.101679  0.008392  0.008392   
CHIP2 POS1 79          -0.051701     0.050221  0.008392  0.008392   
      POS2 159         -0.310623    -0.843624  0.008392  0.008392   
CHIP1 POS2 9